# AMPL Bin Packing Problem with GCG
[![bpp.ipynb](https://img.shields.io/badge/github-%23121011.svg?logo=github)](https://github.com/ampl/colab.ampl.com/blob/master/authors/lentz/gcg/bpp.ipynb) [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ampl/colab.ampl.com/blob/master/authors/lentz/gcg/bpp.ipynb) [![Kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://kaggle.com/kernels/welcome?src=https://github.com/ampl/colab.ampl.com/blob/master/authors/lentz/gcg/bpp.ipynb) [![Gradient](https://assets.paperspace.io/img/gradient-badge.svg)](https://console.paperspace.com/github/ampl/colab.ampl.com/blob/master/authors/lentz/gcg/bpp.ipynb) [![Open In SageMaker Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/ampl/colab.ampl.com/blob/master/authors/lentz/gcg/bpp.ipynb) [![Hits](https://h.ampl.com/https://github.com/ampl/colab.ampl.com/blob/master/authors/lentz/gcg/bpp.ipynb)](https://colab.ampl.com)

Description: Dantzig-Wolfe decomposition for Bin Packing Problem with GCG

Tags: GCG, bpp, amplpy, dantzig-wolfe decomposition, branch-price-and-cut, highlights

Notebook author: Jurgen Lentz <<jurgenlentz26@gmail.com>>

In [1]:
# Install dependencies
%pip install -q amplpy

In [2]:
# Google Colab & Kaggle integration
from amplpy import AMPL, ampl_notebook

ampl = ampl_notebook(
    modules=["gcg", "gurobi", "scip", "highs", "cbc"],  # modules to install
    license_uuid="default",  # license to use
)  # instantiate AMPL object and register magics

## Bin Packing Problem

Given $n$ items with weights $w_{i}$ for all $i \in \{1,...,n\}$ and bins with capacity $C$ (for each bin), the bin packing problem assigns each item $i$ to a bin while minimizing the number of used bins.
BPP can be modeled as follows:

$$
\begin{aligned}
\text{minimize} \quad &\sum_{j = 1}^{m} y_{j} \\
\text{subject to} \quad &\sum_{i = 1}^{n} w_{i} x_{i j} \leq C y_{j} \quad \forall j \in \{1,...,m\} \\
&\sum_{j = 1}^{m} x_{i j} \geq 1 \quad \forall i \in \{1,...,n\} \\
&x_{i j} \in \{0,1\} \quad \forall i \in \{1,...,n\}, j \in \{1,...,m\} \\
&y_{j} \in \{0,1\} \quad \forall j \in \{1,...,m\}
\end{aligned}
$$

We use suffix to feed GCG with a decomposition. Here, we select all allocation constraints to be in the master problem and $m$ knapsack subproblems (GCG aggregates the subproblems).

In [3]:
%%ampl_eval
param n;
param C;

suffix master IN, binary;
suffix block IN, integer;

set I = 1..n ordered;
param w {I} > 0;
param maxVal := max {i in I} w[i];
param maxbins := ceil(n / floor(C / maxVal));

set J = 1..maxbins;

var x {I,J} binary;
var y {J} binary;

minimize Cost:  sum {j in J} y[j];

subject to b_Capacity {j in J}:
   sum {i in I} w[i] * x[i,j] <= C * y[j] suffix block j;

subject to m_Allocate {i in I}:
   sum {j in J} x[i,j] >= 1 suffix master 1;


We generate a small instance with 50 items and bins with a capacity of 120.

In [4]:
ampl.param["n"] = 50
ampl.param["C"] = 120

w = [
    100,
    99,
    98,
    96,
    94,
    90,
    89,
    88,
    88,
    86,
    84,
    81,
    81,
    80,
    79,
    79,
    78,
    76,
    72,
    72,
    72,
    68,
    68,
    65,
    63,
    63,
    63,
    62,
    62,
    57,
    57,
    55,
    48,
    48,
    47,
    45,
    44,
    44,
    41,
    39,
    36,
    33,
    31,
    30,
    28,
    26,
    25,
    24,
    22,
    20,
]

ampl.param["w"] = {i: w[i - 1] for i in range(1, len(w) + 1)}

## Solve with [GCG](https://ampl.com/products/solvers/open-source-solvers/)

In [5]:
ampl.solve(solver="gcg", gcg_options="outlev=1 tech:timing=1")
assert ampl.solve_result == "solved"

GCG 4.0.0:   tech:outlev = 1
  tech:timing = 1
 added complete decomp for original problem with 50 blocks and 50 masterconss, 0 linkingvars, 0 mastervars, and max white score of   0.490000 
presolving:
(round 1, exhaustive) 0 del vars, 0 del conss, 0 add conss, 0 chg bounds, 0 chg sides, 0 chg coeffs, 50 upgd conss, 0 impls, 705 clqs
(round 2, fast)       0 del vars, 0 del conss, 0 add conss, 0 chg bounds, 0 chg sides, 50 chg coeffs, 50 upgd conss, 0 impls, 705 clqs
(round 3, exhaustive) 0 del vars, 0 del conss, 0 add conss, 0 chg bounds, 0 chg sides, 50 chg coeffs, 100 upgd conss, 0 impls, 705 clqs
   (0.1s) probing: 1000/2550 (39.2%) - 0 fixings, 0 aggregations, 9526 implications, 0 bound changes
   (0.1s) probing: 1001/2550 (39.3%) - 0 fixings, 0 aggregations, 9527 implications, 0 bound changes
   (0.1s) probing aborted: 1000/1000 successive useless probings
presolving (4 rounds: 4 fast, 3 medium, 3 exhaustive):
 0 deleted vars, 0 deleted constraints, 0 added constraints, 0 tightene

## Solve with [Gurobi](https://ampl.com/products/solvers/solvers-we-sell/gurobi/)

In [6]:
ampl.solve(solver="gurobi", gurobi_options="outlev=1 tech:timing=1")
assert ampl.solve_result == "solved"

Gurobi 12.0.0: Set parameter LogToConsole to value 1
  tech:outlev = 1
  tech:timing = 1
Set parameter InfUnbdInfo to value 1
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (linux64 - "Ubuntu 22.04.5 LTS")

CPU model: 13th Gen Intel(R) Core(TM) i7-1370P, instruction set [SSE2|AVX|AVX2]
Thread count: 20 physical cores, 20 logical processors, using up to 20 threads

Non-default parameters:
InfUnbdInfo  1

Optimize a model with 100 rows, 2550 columns and 5050 nonzeros
Model fingerprint: 0x9f0306c7
Variable types: 0 continuous, 2550 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

Loaded user MIP start with objective 29

Presolve time: 0.01s
Presolved: 100 rows, 2550 columns, 5050 nonzeros
Variable types: 0 continuous, 2550 integer (2550 binary)

Root relaxation: objective 2.584167e+01, 228 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current

As seen above, GCG keeps up with Gurobi when solving this bin packing instance.

## Solve with [HiGHS](https://ampl.com/products/solvers/open-source-solvers/)

In [7]:
ampl.solve(solver="highs", highs_options="outlev=1 tech:timing=1")
assert ampl.solve_result == "solved"

HiGHS 1.7.1:   tech:outlev = 1
  tech:timing = 1
Running HiGHS 1.7.1 (git hash: dcf3813): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 1e+02]
  Cost   [1e+00, 1e+00]
  Bound  [1e+00, 1e+00]
  RHS    [1e+00, 1e+00]
Assessing feasibility of MIP using primal feasibility and integrality tolerance of       1e-06
Solution has               num          max          sum
Col     infeasibilities      0            0            0
Integer infeasibilities      0            0            0
Row     infeasibilities      0            0            0
Row     residuals            0            0            0
Presolving model
100 rows, 2550 cols, 5050 nonzeros  0s
100 rows, 2550 cols, 5050 nonzeros  0s

MIP start solution is feasible, objective value is 29
Objective function is integral with scale 1

Solving MIP model with:
   100 rows
   2550 cols (2550 binary, 0 integer, 0 implied int., 0 continuous)
   5050 nonzeros

        Nodes      |    B&B Tree     |          

## Solve with [SCIP](https://ampl.com/products/solvers/open-source-solvers/)

In [8]:
ampl.solve(solver="scip", scip_options="outlev=1 tech:timing=1 lim:time=60")
assert ampl.solve_result == "solved"

SCIP 9.0.1:   tech:outlev = 1
  tech:timing = 1
  lim:time = 60
presolving:
(round 1, exhaustive) 0 del vars, 0 del conss, 0 add conss, 0 chg bounds, 0 chg sides, 0 chg coeffs, 50 upgd conss, 0 impls, 705 clqs
(round 2, fast)       0 del vars, 0 del conss, 0 add conss, 0 chg bounds, 0 chg sides, 50 chg coeffs, 50 upgd conss, 0 impls, 705 clqs
(round 3, exhaustive) 0 del vars, 0 del conss, 0 add conss, 0 chg bounds, 0 chg sides, 50 chg coeffs, 100 upgd conss, 0 impls, 705 clqs
   (0.1s) probing: 1000/2550 (39.2%) - 0 fixings, 0 aggregations, 9526 implications, 0 bound changes
   (0.1s) probing: 1001/2550 (39.3%) - 0 fixings, 0 aggregations, 9527 implications, 0 bound changes
   (0.1s) probing aborted: 1000/1000 successive useless probings
   (0.1s) symmetry computation started: requiring (bin +, int +, cont +), (fixed: bin -, int -, cont -)
   (0.1s) symmetry computation finished: 62 generators found (max: 1500, log10 of symmetry group size: 0.0) (symcode time: 0.00)
dynamic symmetry ha

## Solve with [CBC](https://ampl.com/products/solvers/open-source-solvers/)

In [9]:
ampl.solve(solver="cbc", cbc_options="outlev=1 tech:timing=1 lim:time=60")
assert ampl.solve_result == "solved"

cbc 2.10.10:   tech:outlev = 1
  tech:timing = 1
  lim:time = 60
Welcome to the CBC MILP Solver 
Version: 2.10.10 
Build Date: Sep  5 2023 

command line - Cbc_C_Interface -log 1 -solve -quit (default strategy 1)
Continuous objective value is 25.8 - 0.01 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 1 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 1 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 2 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 8 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 7 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 7 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 3 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 2 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 2 strengthened rows, 0 substitutions
Cgl0004I processed model has 100 rows, 2550 columns (2550 inte

GCG outperformes other open-source solvers solving this bin packing instance.